# Lab 4 - Information Processing and Retrieval

## Document organizing, annotation, ranking

In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'rec.autos', 'sci.crypt', 'talk.politics.guns']
collection = fetch_20newsgroups(subset='test', categories=categories)

In [3]:
for i in range(2):
    print(collection.data[i])
    print(collection.target[i])
    print(collection.target_names[collection.target[i]])
    print()

From: holland@CS.ColoState.EDU (douglas craig holland)
Subject: What would happen if export restrictions violated?
Nntp-Posting-Host: beethoven.cs.colostate.edu
Organization: Colorado State University, Computer Science Department
Lines: 15


OK, I heard a lot of talk about the NSA's infamous control over encryption
export through the ITAR.  Here's a question.  Say I develop this great new
encryption system, and I want to sell my software worldwide.  The thought
police then come in and say "This algorithm is a threat to national security.
You will not be permitted to export it."  At this point, what kind of trouble
could I get into if I ignored the ITAR and sold my program to international
customers anyway?

Doug Holland

-- 
|  Doug Holland                | Anyone who tries to take away my freedom  |
|  holland@cs.colostate.edu    | of speech will have to pry it from my     |
|  PGP key available by E-mail | cold, dead lips!!                         |

2
sci.crypt

From: strnlght@netco

Since data is in a text format, we need to process it in order to extract useful information. Let's use TFIDF from scikit-learn

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=False)

X = vectorizer.fit_transform(collection.data)

In [5]:
print(X.shape)

(1545, 26904)


In [6]:
print(vectorizer.get_feature_names_out())

['00' '000' '0000' ... 'zyda' 'zyeh' 'zyxel']


# 1 Cluster the collection

1.1. Extract the vector space of the 20 Newsgroup (use collection.data instruction to ignore
the class variable), and cluster the collection using agglomerative clustering from scikit-learn

In [7]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering().fit(X.toarray())

# Note: we convert X to an array because it's needed by AgglomerativeClustering


In [8]:
print(clustering.labels_)

[0 0 0 ... 0 1 0]


In [9]:
# Print all distinct labels
print(set(clustering.labels_))

{0, 1}


Try to parameterize the number of clusters and the linkage method. 

In [10]:

n_clusters = 20
# choose the best number for n_clusters
single = AgglomerativeClustering(linkage='average', n_clusters=n_clusters, metric='cosine').fit(X.toarray())

complete = AgglomerativeClustering(linkage='average', n_clusters=n_clusters, metric='cosine').fit(X.toarray())

In [ ]:
single_labels = single.fit_predict(X.toarray())
complete_labels = complete.fit_predict(X.toarray())

In [ ]:
print(single_labels)
print(complete_labels)

[0 0 0 ... 0 0 0]
[1 3 1 ... 3 1 3]


1.2. Plot the learned dendogram using the cluster.hierarchy.dendrogram package from scipy.
Compare the clustering solutions produced under single and complete linkage criteria.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

Z = linkage(X.toarray(), 'single', metric='cosine')
plt.figure()
dendrogram(Z, labels=collection.target_names, orientation='right')
plt.xticks(rotation=90)
plt.show()

Z = linkage(X.toarray(), 'complete', metric='cosine')
plt.figure()
dendrogram(Z, labels=collection.target_names, orientation='right')
plt.xticks(rotation=90)
plt.show()



ValueError: Dimensions of Z and labels must be consistent.

<Figure size 640x480 with 0 Axes>

1.3. Evaluate the clustering solution by computing an internal measure (e.g. silhouette) and an
external measure (e.g. adjusted rand index) for the produced clustering solution.

In [ ]:
from sklearn.metrics import silhouette_score, adjusted_rand_score
sscore = silhouette_score(X, single_labels, metric='cosine')
cscore = silhouette_score(X, complete_labels, metric='cosine')

# for the random score we need the true labels
true_labels = collection.target

arsingle = adjusted_rand_score(true_labels, single_labels)
arcomplete = adjusted_rand_score(true_labels, complete_labels)

print(f"Silhouette score for single linkage: {sscore}")
print(f"Silhouette score for complete linkage: {cscore}")
print(f"Adjusted Rand score for single linkage: {arsingle}")
print(f"Adjusted Rand score for complete linkage: {arcomplete}")


Silhouette score for single linkage: 0.26392018268553363
Silhouette score for complete linkage: 0.02735054639193526
Adjusted Rand score for single linkage: -3.059877070379207e-05
Adjusted Rand score for complete linkage: 0.015177362174200237
